In [1]:
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector
from dwave.system import DWaveSampler,EmbeddingComposite